In [1]:
# VENV name: cudaconfig
import torch

print("CUDA is available: ", torch.cuda.is_available())
# True

print("Number of CUDA devices: ", torch.cuda.device_count())
# 5

print("CUDA current device: ", torch.cuda.current_device())
# 0

print("CUDA device name: ", torch.cuda.get_device_name(0))
# 'NVIDIA GeForce RTX 3060's

CUDA is available:  True
Number of CUDA devices:  1
CUDA current device:  0
CUDA device name:  NVIDIA GeForce GTX 1050


In [3]:
import os
import pandas as pd

# Base path to the data folder
base_data_path = 'data'

# List all the folders in the base directory
folders = [f for f in os.listdir(base_data_path) if os.path.isdir(os.path.join(base_data_path, f))]

# Initialize an empty DataFrame to store all the data
combined_df = pd.DataFrame()

# Loop through each folder
for folder in folders:
    folder_path = os.path.join(base_data_path, folder)
    
    # Load the CSV file
    csv_file = os.path.join(folder_path, 'controls.csv')
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
        
        # Add a new column for the image path
        df['image_path'] = df.index.map(lambda i: os.path.join(folder_path, f'frame_{i}.jpg'))
        
        # Combine with the main DataFrame
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# Check the combined DataFrame
print(combined_df.head())


         frame  steering  throttle  brake                 image_path
0  frame_0.jpg         0         1      0  data\bullshit\frame_0.jpg
1  frame_1.jpg         0         1      0  data\bullshit\frame_1.jpg
2  frame_2.jpg         0         1      0  data\bullshit\frame_2.jpg
3  frame_3.jpg         0         1      0  data\bullshit\frame_3.jpg
4  frame_4.jpg         0         1      0  data\bullshit\frame_4.jpg


In [6]:
combined_df


,frame,steering,throttle,brake,image_path
0,frame_0.jpg,0,1,0,data\bullshit\frame_0.jpg
1,frame_1.jpg,0,1,0,data\bullshit\frame_1.jpg
2,frame_2.jpg,0,1,0,data\bullshit\frame_2.jpg
3,frame_3.jpg,0,1,0,data\bullshit\frame_3.jpg
4,frame_4.jpg,0,1,0,data\bullshit\frame_4.jpg
...,...,...,...,...,...
5279,frame_259.jpg,0,1,0,data\training21\frame_435.jpg
5280,frame_260.jpg,0,1,0,data\training21\frame_436.jpg
5281,frame_261.jpg,0,1,0,data\training21\frame_437.jpg
5282,frame_262.jpg,0,1,0,data\training21\frame_438.jpg


In [6]:
from fastai.vision.all import *
import pandas as pd

# Load the labels CSV
labels_df = pd.read_csv('data/test/controls.csv')

# Path to images
path = Path('data/test')

# Define a function to get the control inputs for each image
def get_y(row):
    return tensor([row['steering'], row['throttle'], row['brake']])

def get_x(row):
    return path / row['frame']  # Using '/' for path concatenation

# DataBlock for regression task
dblock = DataBlock(
    blocks=(ImageBlock, RegressionBlock),  # Images as input, continuous labels as output
    get_x=get_x,  # Get image path
    get_y=get_y,  # Get control inputs as a tensor
    splitter=RandomSplitter(valid_pct=0.2),  # Split train/validation
    item_tfms=Resize((398, 94)),  # Resize images
    batch_tfms=aug_transforms()  # Apply augmentations during training
)

# Create DataLoaders
dls = dblock.dataloaders(labels_df)

# Define the learner with a pretrained ResNet model
learn = vision_learner(
    dls, 
    resnet34, 
    loss_func=MSELossFlat(),
    y_range=(-1., 1.)  # Example y_range for normalized outputs; adjust as needed for your task
)

print("Training the model")
# Train the model for a few epochs
learn.fine_tune(10)

# Evaluate the model on the validation set
print("Evaluating the model")
learn.validate()

# Save the entire model for future use
learn.export('trackmania_learner.pkl')

Training the model


epoch,train_loss,valid_loss,time
0,0.728773,0.585586,00:45


epoch,train_loss,valid_loss,time
0,0.638688,0.455145,01:01
1,0.628212,0.424364,01:06
2,0.604955,0.305409,01:06
3,0.585382,0.444560,01:06
4,0.547881,0.322901,01:02
5,0.516271,0.280230,01:00
6,0.485756,0.270969,01:00
7,0.455555,0.251855,01:00
8,0.432079,0.247618,01:01
9,0.415165,0.234608,01:00


Evaluating the model
